In [1]:
from DBEngine import db_engine

In [9]:
def parse_entry(entry):
  o = dict()
  o['id'] = entry['_id']
  o['amenities'] = entry['amenityIds']
  o['brand'] = entry['brandCode']
  o['name'] = entry['name']
  o['address'] = entry['address']['addressLine1']
  o['city'] = entry['address']['city']
  o['country'] = entry['address']['country']
  o['state'] = entry['address']['state']
  o['lat'] = entry['localization']['coordinate']['latitude']
  o['lon'] = entry['localization']['coordinate']['longitude']
  o['master_image_text'] = entry['masterImage']['altText']
  o['master_image'] = entry['masterImage']['variants'][2]['url']
  o['images'] = [x['variants'][2]['url'] for x in entry['images']['carousel']]
  if entry['tripAdvisorLocationSummary'] is not None:
    o['review_number'] = entry['tripAdvisorLocationSummary']['numReviews']
    o['review_avg'] = entry['tripAdvisorLocationSummary']['rating']
  else:
    o['review_number'] = None
    o['review_avg'] = None

  if entry['leadRate'] is not None:
    o['disaplay_price'] = entry['leadRate']['lowest']['rateAmount']
    if entry['leadRate']['hhonors'] is not None:
      o['max_price'] = entry['leadRate']['hhonors']['max']['rateAmount']
      o['min_price'] = entry['leadRate']['hhonors']['min']['rateAmount']
    else:
      o['max_price'] = None
      o['min_price'] = None
  else:
    o['disaplay_price'] = None
    o['max_price'] = None
    o['min_price'] = None

  return o

In [6]:
import json

def load_batch(n):
  f = open(f"responses/r{n}.json")
  data = json.load(f)

  data = data["data"]["hotelSummaryOptions"]["hotels"]

  entries = [parse_entry(e) for e in data]

  def command(cursor): 
    cursor.executemany("insert into hotels values (%(id)s, %(amenities)s, %(brand)s, %(name)s, %(address)s, %(city)s, %(country)s, %(state)s, ST_SetSRID(ST_Point(%(lon)s,%(lat)s),4326), %(master_image_text)s, %(master_image)s, %(images)s, %(review_number)s, %(review_avg)s, %(disaplay_price)s, %(max_price)s, %(min_price)s)", entries)
  
  db_engine.execute(command)
  db_engine.commit()

In [10]:
for i in range(0, 16):
  print(i)
  load_batch(i)

15
